# PREPARATIVOS

In [3]:
import pandas as pd
import numpy as np
import itertools as it
import re
import os
from funciones import *

In [4]:
FBL3N = leer_excel("FBL3N.xlsx")

Leyendo archivo Excel: FBL3N.xlsx
Archivo Excel leído.


In [5]:
FBL5N = leer_excel("FBL5N.xlsx")

Leyendo archivo Excel: FBL5N.xlsx
Archivo Excel leído.


In [6]:
KNA1 = leer_excel("KNA1.xlsx")

Leyendo archivo Excel: KNA1.xlsx
Archivo Excel leído.


In [7]:
#PROPUESTA = leer_excel("PROPUESTA.xlsx", None)

# PARTE 1

In [8]:
# PARTE 1

textos = FBL3N["Texto"]
#FBL3N["Categoría"] = textos.apply(detectar_categoria)
FBL3N["RUT"] = textos.apply(detectar_rut)
FBL3N["N° Factura"] = textos.apply(detectar_n_factura)
#FBL3N["Números"] = textos.apply(detectar_numeros)

RUT_FBL3N = FBL3N[pd.notna(FBL3N["RUT"])]
FACT_FBL3N = FBL3N[pd.isna(FBL3N["RUT"]) & pd.notna(FBL3N["N° Factura"])]
#con_otros_numeros = FBL3N[pd.isna(FBL3N["RUT"]) & pd.isna(FBL3N["N° Factura"]) & pd.notna(FBL3N["Números"])]
#sin_numero = FBL3N[pd.isna(FBL3N["RUT"]) & pd.isna(FBL3N["N° Factura"]) & pd.isna(FBl3N["Números"])]

"""
try:
	os.mkdir("FBL3N/")
except FileExistsError:
	pass

for dataframe, nombre in ((FBL3N, "NUEVO"), (con_rut, "RUT"), (con_n_factura, "N_FACTURA"), (con_otros_numeros, "OTROS_N"), (sin_numero, "SIN_N")):
	escribir_excel(dataframe, "FBL3N/" + nombre + ".xlsx")
"""

'\ntry:\n\tos.mkdir("FBL3N/")\nexcept FileExistsError:\n\tpass\n\nfor dataframe, nombre in ((FBL3N, "NUEVO"), (con_rut, "RUT"), (con_n_factura, "N_FACTURA"), (con_otros_numeros, "OTROS_N"), (sin_numero, "SIN_N")):\n\tescribir_excel(dataframe, "FBL3N/" + nombre + ".xlsx")\n'

# PARTE 2

### Trabajar con calces de factura

In [9]:
FBL5N["Referencia"] = FBL5N["Referencia"].apply(lambda s: str(s))
FBL5N["Cuenta"] = FBL5N["Cuenta"].apply(lambda n: str(n))

KNA1["Nº ident.fis.1"] = KNA1["Nº ident.fis.1"].apply(lambda s: str(s).lstrip("0"))
KNA1["Cliente"] = KNA1["Cliente"].apply(lambda s: str(s))

In [10]:
# Calzar números de factura
filtro_fact = FBL5N["Referencia"].isin(FACT_FBL3N["N° Factura"])
FACT_FBL5N = FBL5N[filtro_fact]
NO_FACT_FBL5N = FBL5N[~filtro_fact]

In [11]:
# Calzar RUTs
ID_FBL3N = pd.merge(RUT_FBL3N, KNA1[["Nº ident.fis.1", "Cliente"]], left_on="RUT", right_on="Nº ident.fis.1")

filtro_id = NO_FACT_FBL5N["Cuenta"].isin(ID_FBL3N["Cliente"])
ID_FBL5N = NO_FACT_FBL5N[filtro_id]
NN_FBL5N = NO_FACT_FBL5N[~filtro_id]

In [12]:
#escribir_excel(ID_FBL3N, "ID_FBL3N.xlsx")

In [13]:
#escribir_excel(MATCHES_ID, "MATCHES_ID.xlsx")

In [14]:
#escribir_excel(MATCHES_FACT, "MATCHES_FACT.xlsx")

In [15]:
try:
    os.mkdir("RESULTADOS/")
except:
    pass

In [16]:
MATCHES_FACT = pd.merge(FACT_FBL3N, FACT_FBL5N, left_on="N° Factura", right_on="Referencia")
escribir_excel(FACT_FBL5N, "RESULTADOS/N_FACTURAS.xlsx")

Escribiendo nuevo archivo Excel: RESULTADOS/N_FACTURAS.xlsx
Archivo Excel escrito.


# PARTE 3

### Trabajar con calces de ID

In [17]:
MATCHES_ID = pd.merge(ID_FBL3N, ID_FBL5N, left_on="Cliente", right_on="Cuenta", how="left")

In [25]:
def sumas_parciales(serie):
    retorno = []
    for largo in range(min(serie.size, 15)):
        comb = np.array(list(it.combinations(serie, largo)))
        retorno.append(comb.sum())
        
    return pd.Series(retorno)

In [19]:
def calzar(fila, FBL5N):
    monto = -fila["Importe en moneda local"]
    ID = fila["Cliente"]
    
    partidas = FBL5N[FBL5N["Cuenta"] == ID]
    n = partidas["Nｺ doc."].size
    
    if monto in sumas_parciales(partidas):
        return True
    return False

In [20]:
MATCHES_ID

,Icono part.abiertas/comp.,Asignación,Nº documento,División,Clase de documento,Fecha de documento,Clave contabiliz.,Importe en moneda local,Moneda local,Indicador impuestos,...,Importe en ML,CPag,Texto_y,nan,NaT,Clave ref.1,Clave ref.2,VP,Lib.mayor,Usuario
0,NaN,0.000000e+00,100133490.0,NaN,CT,2020-04-06,50.0,-82394,CLP,NaN,...,-23448.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1.104011e+09,GVERGARA
1,NaN,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,...,-1.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1.104011e+09,JFABRES
2,NaN,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,...,-1104.0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1.104011e+09,JFABRES
3,NaN,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,...,-1.0,NaN,ABONO SEGUN FACTURA 8913231,NaN,NaT,NaN,NaN,NaN,1.104011e+09,VPADRON
4,NaN,0.000000e+00,100189371.0,NaN,CT,2020-04-30,50.0,-33184,CLP,NaN,...,-1.0,NaN,ABONO SEGUN FACTURA 8935682,NaN,NaT,NaN,NaN,NaN,1.104011e+09,VPADRON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25846,NaN,2.021102e+14,100389327.0,NaN,CT,2021-10-22,50.0,-9420739,CLP,NaN,...,3034988.0,Z030,NaN,NaN,NaT,Riesgo Empre,1341601,F,1.104011e+09,SD060
25847,NaN,2.021102e+14,100389327.0,NaN,CT,2021-10-22,50.0,-9420739,CLP,NaN,...,95712.0,Z030,NaN,NaN,NaT,Riesgo Empre,1341601,F,1.104011e+09,SD060
25848,NaN,2.021102e+14,100389327.0,NaN,CT,2021-10-22,50.0,-9420739,CLP,NaN,...,1868693.0,Z030,NaN,NaN,NaT,Riesgo Empre,1341601,F,1.104011e+09,SD060
25849,NaN,2.021102e+14,100389327.0,NaN,CT,2021-10-22,50.0,-9420739,CLP,NaN,...,676634.0,Z030,NaN,NaN,NaT,Riesgo Empre,1341601,F,1.104011e+09,SD060


In [22]:
grupos = MATCHES_ID.groupby(["Nº documento", "Cliente"])

In [ ]:
lista = []

for indice, grupo in grupos:
    sumas = sumas_parciales(grupo["Importe en ML"])
    resultado = grupo[grupo["Importe en moneda local"].isin(sumas)]
    lista.append(resultado)

pd.concat(lista)

In [ ]:
COPIA_ID_FBL5N = ID_FBL5N.copy()
COPIA_ID_FBL5N = COPIA_ID_FBL5N.sort_values(by="Demora", ascending=False)
#MATCHES_ID_1_N = pd.concat(ID_FBL3N.apply(calzar, args=(COPIA_ID_FBL5N,), axis=1).to_numpy())
#MATCHES_ID_1_N

In [ ]:
#escribir_excel(MATCHES_ID_1_N, "MATCHES_ID_1-N.xlsx")

In [ ]:
# TODO: Hay un dato duplicado, hay que corregir eso
MATCHES_ID_1_1 = MATCHES_ID[MATCHES_ID["Importe en moneda local"] == -MATCHES_ID["Importe en ML"]]

In [ ]:
MATCHES_ID_1_1 = MATCHES_ID_1_1.sort_values(by="Demora", ascending=False)
MATCHES_ID_1_1 = MATCHES_ID_1_1.drop_duplicates(subset=["Nº documento"])

escribir_excel(MATCHES_ID_1_1, "MATCHES_ID_1-1.xlsx")

# PARTE 4

### Determinar qué datos encontré yo que Roberto no, y viceversa

1. RUT y Monto

In [ ]:
RUT_Y_MONTO = leer_excel("002_RutyMonto.xlsx")

In [ ]:
outer_rym = pd.merge(MATCHES_ID_1_1, RUT_Y_MONTO, on="Nº documento", how="outer", indicator=True)

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "left_only"], "RUT_Y_MONTO_LEFT.xlsx")

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "right_only"], "RUT_Y_MONTO_RIGHT.xlsx")

2. Solo RUT

In [ ]:
SOLO_RUT = leer_excel("002_SoloRut.xlsx")

In [ ]:
outer_sr = pd.merge(MATCHES_ID, SOLO_RUT, on="Nº documento", how="outer", indicator=True)

In [ ]:
escribir_excel(outer_sr[outer_sr["_merge"] == "left_only"], "SOLO_RUT_LEFT.xlsx")

In [ ]:
escribir_excel(outer_sr[outer_sr["_merge"] == "right_only"], "SOLO_RUT_RIGHT.xlsx")